In [75]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import sys

In [76]:
!pip install folium
import folium
# import k-means from clustering stage
from sklearn.cluster import KMeans
import urllib, json

In [77]:
##SCRAPPING DISTRICS

#define headers for the request
headers = requests.utils.default_headers()
headers.update({"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36"})

#Wikipedia link for the information to scrap
url = "https://en.wikipedia.org/wiki/Districts_of_Madrid"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

In [78]:
#Finding columns name html style
columns_name = str(soup.center.table.tbody.tr).replace('\n','')

#cleaning tags
columns_string=re.findall('<th>(.+?)<',columns_name)

columns = [list(columns_string)[1],list(columns_string)[3]]
#Transform in a Series to create data frame
columns_name = pd.Series(list(columns))

#finding dataframe values
values = str(soup.center.find_all("td")).replace('\n','')
values = re.findall('>(.+?)</td>',values)

#finding Neighbourhood clean name
neigh = []
for value in values:
    if len(re.findall('>(.+?)</a>',str(value)))<1:
        neigh.append(value)
    else:
        a = re.findall('>(.+?)<',str(value)) #result type is a list
        neigh.append(a[0])

In [79]:
#Define two empty list to get District Name and population
d=[]
h1=[]
h2=[] #h1 will return a nest list, to extract the population in just one list
i=0

for i in range(int(len(neigh)/7)):
    if i==0:
        d.append(neigh[i+1])
        h1.append(re.findall('>(.+?)</span>',str(neigh[i+3])))
        h2.append(int(str(h1[i][i]).replace(',','')))
    else:
        d.append(neigh[i*7+1])
        h1.append(re.findall('>(.+?)</span>',neigh[i*7+3]))
        h2.append(int(str(h1[i][0]).replace(',','')))

        #Creating the df wich contains info about Madrid districts

dicti = {'District Name':d,
         'District Population':h2    
}

mddist = pd.DataFrame(dicti)
mddist


,District Name,District Population
0,Centro,131928
1,Arganzuela,151965
2,Retiro,118516
3,Salamanca,143800
4,Chamartín,143424
5,Tetuán,153789
6,Chamberí,137401
7,Fuencarral-El Pardo,238756
8,Moncloa-Aravaca,116903
9,Latina,233808


In [80]:
#Conecting with Comunidad de Madrid official data to get neighborhoods

#Wikipedia link for the information to scrap
url2 = "https://datos.comunidad.madrid/catalogo/dataset/91857345-056b-4aa7-96f5-6310dac619bf/resource/b9cf6d43-79b0-400e-82de-ca00de21f6bb/download/barrios_municipio_madrid.json"
req2 = requests.get(url2, verify=False)
#With the mdjson and District name We can find the Neighborhood
mdjson = req2.json()

mdjson
#mdjson has spaces leading and ending in barrio_nombre and distrito_nombre, let's clean that


for i in range(len(mdjson['data'])):
        for j in mdjson['data'][i]:
            if type(mdjson['data'][i][j])==str:
                mdjson['data'][i][j]=mdjson['data'][i][j].strip()


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [81]:
#Now we create 2 empty list, to append distrito_nombre(District Name) & barrio_nombre(Neighborhood) 
#also we use madrid['District Name'] to go through the json


d = [] #district
n = [] #neighborhood
i=0
j=0

for j in range (len(mdjson['data'])):
    if mdjson['data'][j]['distrito_nombre']=='Tetuan':
        d.append('Tetuán')
        n.append(mdjson['data'][j]['barrio_nombre'])
    else:
        d.append(mdjson['data'][j]['distrito_nombre'])
        n.append(mdjson['data'][j]['barrio_nombre'])

#lets create a 2nd df with the new info

mdneigh = pd.DataFrame({'District Name':d,
                       'Neighborhood':n
})
mdneigh

,District Name,Neighborhood
0,Centro,Palacio
1,Centro,Embajadores
2,Centro,Cortes
3,Centro,Justicia
4,Centro,Universidad
5,Centro,Sol
6,Arganzuela,Imperial
7,Arganzuela,Las Acacias
8,Arganzuela,La Chopera
9,Arganzuela,Legazpi


In [82]:
#Now lets join df madrid with mdneigh 
madrid = pd.merge(mdneigh,mddist,on='District Name')
madrid.head(10)
#Extract data from RentaxNeighborhood.csv from github repository
madridrent = pd.read_csv('https://raw.githubusercontent.com/cazuaje27/BattleofNeighborhoods/master/RentaxNeighborhood.csv', sep=';',skiprows=2,header=1)
madridrent.rename(columns={'Territorio':'Neighborhood','Renta neta media por hogar':'Net Income by Home'}, inplace=True)


#We need to change number format . as thousand separator
#for i in range(len(madridrent['Net Income by Home'])):
    #if madridrent['Net Income by Home'][i]==0:
        #madridrent['Net Income by Home'][i]=int(0)
   # else:
      #  madridrent['Net Income by Home'][i]=int(str(madridrent['Net Income by Home'][i]).replace('.',''))

#Merge to get all Rent in madrid dataframe

madrid = madrid.merge(madridrent, on=['Neighborhood'])
madrid.drop(axis=1, columns=['Orden','Cod. Territorio'], inplace=True)
madrid.head(10)

,District Name,Neighborhood,District Population,Net Income by Home
0,Centro,Palacio,131928,34.676
1,Centro,Embajadores,131928,26.000
2,Centro,Cortes,131928,34.953
3,Centro,Justicia,131928,40.315
4,Centro,Universidad,131928,30.702
5,Centro,Sol,131928,30.936
6,Arganzuela,Imperial,151965,41.951
7,Arganzuela,Las Acacias,151965,44.669
8,Arganzuela,La Chopera,151965,31.934
9,Arganzuela,Legazpi,151965,48.935


In [83]:
#Let's read the house price for m2 in Madrid by Neighborhood
#ph = price house
ph = pd.read_excel('https://github.com/cazuaje27/BattleofNeighborhoods/raw/master/HousingPrice2019.xls',sep='/t', skiprows=7)
ph.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
ph.rename(columns={'Unnamed: 1':'Neighborhood', 'Unnamed: 2':'House Price 2019 €/m2','Unnamed: 3':'House Price 2018 €/m2'}, inplace=True)
ph.dropna(inplace=True)
ph['Neighborhood']=ph['Neighborhood'].str.strip().str.strip('0123456789.').str.strip()

#Now merge the data with madrid df
madrid = madrid.merge(ph, on=['Neighborhood'], how='inner')

In [130]:
#Reading the Madrid geojson to make a map using folium

urlgeojson=('https://raw.githubusercontent.com/cazuaje27/BattleofNeighborhoods/master/Madrid.geojson')
mdgeojson = requests.get(urlgeojson).json()

#Lets check if the District & Neighborhood have the same format that we already have
d2=[]
n2=[]
for i in range(len(mdgeojson['features'])):
    d2.append(mdgeojson['features'][i]['properties']['NOMDIS'])
    n2.append(mdgeojson['features'][i]['properties']['Name'])

mdmap=pd.DataFrame({'District Name':d2,
                    'Neighborhood':n2
                   })
mdmap.describe()

,District Name,Neighborhood
count,131,131
unique,21,131
top,Ciudad Lineal,Adelfas
freq,9,1


In [131]:
#Lets change format for District Name
#first we define two series with the unique values of each column
mapdist = mdmap['District Name'].sort_values().unique()
maddist = madrid['District Name'].sort_values().unique()
#empty list for district name corrected
d3=[]

for j,k in zip(range(len(mapdist)),range(len(maddist))):
    for i in range(len(mdgeojson['features'])):
        if mdgeojson['features'][i]['properties']['NOMDIS']==mapdist[j]:
            mdgeojson['features'][i]['properties']['NOMDIS']=maddist[k]

#fill the empty list with the right names                   
for i in range(len(mdgeojson['features'])):
    d3.append(mdgeojson['features'][i]['properties']['NOMDIS'])

mdmap = pd.DataFrame({'District Name':d3,
                    'Neighborhood':n2
                   })


In [132]:
# The code was removed by Watson Studio for sharing.

In [133]:
#Now let use google cloud geocoding API to get coordinates from Madrid and its Neighborhood
#Define url to make the request
geocoding_url="https://maps.googleapis.com/maps/api/geocode/json?address="
consult_madrid = "{city},+Spain&key={key}".format(city='Madrid',key=ApiKey)
geocoding_madrid = requests.get(geocoding_url+consult_madrid)
#Json results
geocodingjson = geocoding_madrid.json()
#Now we extract latitude and longitude from Madrid to make the Map using folium
mdlat = geocodingjson['results'][0]['geometry']['location']['lat']
mdlng = geocodingjson['results'][0]['geometry']['location']['lng']

In [134]:

consult_neigh = "{neigh},+{district},+Madrid,+Spain&key={key}".format(neigh=madrid['Neighborhood'][2], district=madrid['District Name'][2], key=ApiKey)
geocoding_neigh = requests.get(geocoding_url+consult_neigh)
geoneighjson=geocoding_neigh.json()
neighlat = geoneighjson['results'][0]['geometry']['location']['lat']
neighlng = geoneighjson['results'][0]['geometry']['location']['lng']


In [135]:
#Finding every Neighborhood coordinates for the maps.

neigh_lat = []
neigh_lon = []

for i in range(len(madrid['Neighborhood'])):
    consult_neigh = "{neigh},+{district},+Madrid,+Spain&key={key}".format(neigh=madrid['Neighborhood'][i], district=madrid['District Name'][i], key=ApiKey)
    geocoding_neigh = requests.get(geocoding_url+consult_neigh)
    geoneighjson=geocoding_neigh.json()
    neigh_lat.append(geoneighjson['results'][0]['geometry']['location']['lat'])
    neigh_lon.append(geoneighjson['results'][0]['geometry']['location']['lng'])
    
    
madrid['Neighborhood Latitude']=neigh_lat
madrid['Neighborhood Longitude']=neigh_lon

,District Name,Neighborhood,District Population,Net Income by Home,House Price 2019 €/m2,House Price 2018 €/m2,Neighborhood Latitude,Neighborhood Longitude
0,Centro,Palacio,131928,34.676,4852.0,4873.0,40.416339,-3.714318
1,Centro,Embajadores,131928,26.000,4479.0,4239.0,40.408129,-3.707739
2,Centro,Cortes,131928,34.953,5272.0,5106.0,40.414221,-3.699941
3,Centro,Justicia,131928,40.315,5893.0,5989.0,40.424531,-3.699293
4,Centro,Universidad,131928,30.702,5282.0,5068.0,40.423372,-3.707792
5,Centro,Sol,131928,30.936,5110.0,5263.0,40.416663,-3.704169
6,Arganzuela,Imperial,151965,41.951,3922.0,3896.0,40.406845,-3.720088
7,Arganzuela,Las Acacias,151965,44.669,4188.0,4046.0,40.403926,-3.706549
8,Arganzuela,La Chopera,151965,31.934,3625.0,3727.0,40.394836,-3.699599
9,Arganzuela,Legazpi,151965,48.935,4394.0,4263.0,40.389234,-3.686412


In [137]:
#Making the first Choropleth Map for the Analysis District Population in Madrid
madridmap = folium.Map(location=[mdlat,mdlng], zoom_start=11)

madridPopulation = folium.Choropleth(geo_data=mdgeojson,
                  name='choropleth',
                  data=madrid,
                  columns=['District Name', 'District Population'],
                  key_on='feature.properties.NOMDIS',
                  fill_color='YlOrRd',
                  fill_opacity=0.5,
                  line_opacity=0.2,
                  legend_name='District Population',
                  threshold_scale = [0,50000,100000,150000,200000,250000]

)


madridPopulation.add_to(madridmap)
madridPopulation.geojson.add_child(folium.features.GeoJsonTooltip(['Name','NOMDIS'],labels=False))
madridmap

In [138]:
#2nd Map for the Analysis, Net Income per Neighborhood

madridmap = folium.Map(location=[mdlat,mdlng], zoom_start=11)
madridNetIncome = folium.Choropleth(geo_data=mdgeojson,
                  name='choropleth',
                  data=madrid,
                  columns=['Neighborhood', 'Net Income by Home'],
                  key_on='feature.properties.Name',
                  fill_color='BuGn',
                  nan_fill_color = 'white',                  
                  fill_opacity=0.7,
                  line_opacity=0.5,
                  legend_name='Net Income by Home (x1000)',
                  threshold_scale = [0,20,40,60,80,100,120]
                  

)

madridNetIncome.add_to(madridmap)
madridNetIncome.geojson.add_child(folium.features.GeoJsonTooltip(['Name','NOMDIS'],labels=False))
madridmap

In [139]:
#3rd Map for the Analysis, House Price in 2019 €/m2

madridmap = folium.Map(location=[mdlat,mdlng], zoom_start=11)
madridHouse2019 = folium.Choropleth(geo_data=mdgeojson,
                  name='choropleth',
                  data=madrid,
                  columns=['Neighborhood', 'House Price 2019 €/m2'],
                  key_on='feature.properties.Name',
                  fill_color='PuBuGn',
                  nan_fill_color = 'white',                  
                  fill_opacity=0.7,
                  line_opacity=0.5,
                  legend_name='House Price in 2019 Euro/m2',
                  threshold_scale = [0,1000,2000,3000,4000,5000,6000,7500,8500]
)

madridHouse2019.add_to(madridmap)
madridHouse2019.geojson.add_child(folium.features.GeoJsonTooltip(['Name','NOMDIS'],labels=False))
madridmap

In [147]:
# The code was removed by Watson Studio for sharing.

In [163]:
#Now start the neighborhood investigation using Foursquare

#Funtion to scrap the foursquare information, I going to include PostalCode and Borough. Because many neighborhood has the same location of one PostalCode.

LIMIT = 100

def getNearbyVenues(district, neigh, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for district, neigh, lat, lng in zip(district, neigh, latitudes, longitudes):
        #print('{}: {}'.format(district,neigh))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            district,
            neigh,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District Name',
                  'Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [174]:
madrid_venues = getNearbyVenues(madrid['District Name'], madrid['Neighborhood'], madrid['Neighborhood Latitude'], madrid['Neighborhood Longitude'], radius=500)
madrid_venues.groupby(['District Name','Neighborhood']).count()

Latitude  Longitude  Venue  \
District Name       Neighborhood                                               
Arganzuela          Atocha                              40         40     40   
                    Delicias                            38         38     38   
                    Imperial                            24         24     24   
                    La Chopera                          42         42     42   
                    Las Acacias                         40         40     40   
                    Legazpi                             20         20     20   
                    Palos de Moguer                     54         54     54   
Barajas             Aeropuerto                           6          6      6   
                    Alameda de Osuna                    11         11     11   
                    Casco Histórico de Barajas          40         40     40   
                    Corralejos                           8          8      8   
Carabanchel         Abrantes                             2          2      2   
                    Buenavista                           5          5      5   
                    Comillas                            14         14     14   
                    Opañel                              12         12     12   
                    Puerta Bonita                        4          4      4   
                    San Isidro                           3          3      3   
                    Vista Alegre                        17         17     17   
Centro              Cortes                             100        100    100   
                    Embajadores                         47         47     47   
                    Justicia                           100        100    100   
                    Palacio                             69         69     69   
                    Sol                                 81         81     81   
                    Universidad                        100        100    100   
Chamartín           Castilla                            22         22     22   
                    Ciudad Jardín                       46         46     46   
                    El Viso                             35         35     35   
                    Hispanoamérica                      95         95     95   
                    Nueva España                        48         48     48   
                    Prosperidad                         80         80     80   
...                                                    ...        ...    ...   
San Blas-Canillejas Amposta                              9          9      9   
                    Arcos                               12         12     12   
                    Canillejas                          10         10     10   
                    El Salvador                         12         12     12   
                    Hellín                               4          4      4   
                    Rejas                                8          8      8   
                    Rosas                                9          9      9   
                    Simancas                            40         40     40   
Tetuán              Almenara                            10         10     10   
                    Bellas Vistas                       39         39     39   
                    Berruguete                          19         19     19   
                    Castillejos                         84         84     84   
                    Cuatro Caminos                      68         68     68   
                    Valdeacederas                        5          5      5   
Usera               Almendrales                         18         18     18   
                    Moscardó                            15         15     15   
                    Orcasitas                            9          9      9   
                    Orcasur                       

In [176]:
#Now we create a dataframe with the venues as columns
madrid_venuescat = pd.get_dummies(madrid_venues[['Venue Category']], prefix="", prefix_sep="")

# adding District Name and Neighborhood column back to dataframe
#We change the name of the columns neighborhood to neighbohoods because there is one venue with the same name
madrid_venuescat[['District Name','Neighborhoods']] = madrid_venues[['District Name','Neighborhood']] 

# move District Name and Neighborhood column to the first column
fixed_columns = [madrid_venuescat.columns[-2]]+ [madrid_venuescat.columns[-1]] +list(madrid_venuescat.columns[:-2])
madrid_venuescat = madrid_venuescat[fixed_columns]

madrid_venuescat.head()

,District Name,Neighborhoods,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Centro,Palacio,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Centro,Palacio,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Centro,Palacio,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Centro,Palacio,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Centro,Palacio,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [178]:
madrid_grouped = madrid_venuescat.groupby(['District Name','Neighborhoods']).mean().reset_index()
madrid_grouped

,District Name,Neighborhoods,Accessories Store,Adult Boutique,Airport Terminal,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Arganzuela,Atocha,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
1,Arganzuela,Delicias,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.026316,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
2,Arganzuela,Imperial,0.0,0.00,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
3,Arganzuela,La Chopera,0.0,0.00,0.000000,0.000000,0.00,0.023810,0.047619,0.023810,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
4,Arganzuela,Las Acacias,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.075000,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.025000,0.0,0.000000
5,Arganzuela,Legazpi,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
6,Arganzuela,Palos de Moguer,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.074074,0.000000,...,0.0,0.018519,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
7,Barajas,Aeropuerto,0.0,0.00,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
8,Barajas,Alameda de Osuna,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.000000,0.000000,0.0,0.000000
9,Barajas,Casco Histórico de Barajas,0.0,0.00,0.000000,0.000000,0.00,0.050000,0.000000,0.000000,...,0.0,0.000000,0.00,0.0,0.00,0.0,0.025000,0.000000,0.0,0.000000


In [184]:
#Lets create a funtion to get the most common venues for each group.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [185]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District Name','Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted[['District Name','Neighborhoods']] = madrid_grouped[['District Name','Neighborhoods']]

for ind in np.arange(madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,District Name,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arganzuela,Atocha,Platform,Restaurant,Spanish Restaurant,Train Station,Breakfast Spot,Hotel,Grocery Store,Museum,Music Venue,Science Museum
1,Arganzuela,Delicias,Tapas Restaurant,Spanish Restaurant,Beer Garden,Restaurant,Bakery,Gym / Fitness Center,Burger Joint,Gym,Mediterranean Restaurant,Grocery Store
2,Arganzuela,Imperial,Spanish Restaurant,Grocery Store,Spa,Gym Pool,Park,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Café,Burger Joint
3,Arganzuela,La Chopera,Italian Restaurant,Tapas Restaurant,Park,Spanish Restaurant,Burger Joint,Art Gallery,Coffee Shop,Shopping Mall,Gastropub,Pet Store
4,Arganzuela,Las Acacias,Spanish Restaurant,Art Gallery,Pizza Place,Park,Bar,Bakery,Grocery Store,Tapas Restaurant,Sushi Restaurant,Theme Restaurant
5,Arganzuela,Legazpi,Spanish Restaurant,Bar,Dessert Shop,Park,Supermarket,Restaurant,Coffee Shop,Multiplex,General Entertainment,Tapas Restaurant
6,Arganzuela,Palos de Moguer,Spanish Restaurant,Art Gallery,Tapas Restaurant,Beer Garden,Gym / Fitness Center,Plaza,Pizza Place,Burger Joint,Restaurant,Park
7,Barajas,Aeropuerto,Spanish Restaurant,Hotel Bar,Airport Terminal,Hotel,Food Court,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food
8,Barajas,Alameda de Osuna,Spanish Restaurant,Tennis Court,Scenic Lookout,Chinese Restaurant,Bar,Bakery,Restaurant,Travel Agency,Gym / Fitness Center,Smoke Shop
9,Barajas,Casco Histórico de Barajas,Hotel,Spanish Restaurant,Restaurant,Tapas Restaurant,Argentinian Restaurant,Plaza,Mexican Restaurant,Bistro,Metro Station,Boarding House


In [198]:
#Lets Create Cluster Using information 
kclusters = 5

madrid_grouped_clustering = madrid_grouped.drop(['Neighborhoods','District Name'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(madrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 4, 4, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 3, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       3, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0], dtype=int32)

In [239]:

#Lets include the Clusters with all the information
madrid_merged = madrid[['District Name','Neighborhood','Neighborhood Latitude','Neighborhood Longitude']]
madrid_merged.drop(axis=0, index=126,inplace=True)

# add clustering labels
madrid_merged['Cluster Labels'] = kmeans.labels_



# merge madrid_grouped with madrid_data to add latitude/longitude for each neighborhood
madrid_merged = madrid_merged.join(neighborhoods_venues_sorted.set_index(['District Name','Neighborhoods']), on=['District Name','Neighborhood'])

madrid_merged.head() # check the last columns!

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,District Name,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,Palacio,40.416339,-3.714318,0,Plaza,Spanish Restaurant,Tapas Restaurant,Restaurant,Mexican Restaurant,Historic Site,Bar,Ramen Restaurant,Beer Bar,Music Venue
1,Centro,Embajadores,40.408129,-3.707739,0,Tapas Restaurant,Bar,Spanish Restaurant,Coffee Shop,Plaza,Art Gallery,Italian Restaurant,Outdoors & Recreation,Bistro,Café
2,Centro,Cortes,40.414221,-3.699941,0,Bar,Hotel,Hostel,Restaurant,Theater,Spanish Restaurant,Plaza,Tapas Restaurant,Mediterranean Restaurant,Café
3,Centro,Justicia,40.424531,-3.699293,0,Restaurant,Hotel,Spanish Restaurant,Bakery,Café,Mediterranean Restaurant,Bar,Pizza Place,Vegetarian / Vegan Restaurant,Bookstore
4,Centro,Universidad,40.423372,-3.707792,0,Hotel,Spanish Restaurant,Tapas Restaurant,Café,Restaurant,Cocktail Bar,Hostel,Theater,Bookstore,Argentinian Restaurant


Arganzuela Atocha 40
Arganzuela Delicias 38
Arganzuela Imperial 24
Arganzuela La Chopera 42
Arganzuela Las Acacias 40
Arganzuela Legazpi 20
Arganzuela Palos de Moguer 54
Barajas Aeropuerto 6
Barajas Alameda de Osuna 11
Barajas Casco Histórico de Barajas 40
Barajas Corralejos 8
Barajas Timón 2
Carabanchel Abrantes 5
Carabanchel Buenavista 14
Carabanchel Comillas 12
Carabanchel Opañel 4
Carabanchel Puerta Bonita 3
Carabanchel San Isidro 17
Carabanchel Vista Alegre 100
Centro Cortes 47
Centro Embajadores 100
Centro Justicia 69
Centro Palacio 81
Centro Sol 100
Centro Universidad 22
Chamartín Castilla 46
Chamartín Ciudad Jardín 35
Chamartín El Viso 95
Chamartín Hispanoamérica 48
Chamartín Nueva España 80
Chamartín Prosperidad 100
Chamberí Almagro 60
Chamberí Arapiles 100
Chamberí Gaztambide 100
Chamberí Ríos Rosas 100
Chamberí Trafalgar 59
Chamberí Vallehermoso 15
Ciudad Lineal Atalaya 23
Ciudad Lineal Colina 17
Ciudad Lineal Concepción 18
Ciudad Lineal Costillares 16
Ciudad Lineal Pueblo N

In [231]:
zz = pd.Series(madrid_venues['Neighborhood'].unique())
zz.sort_values()
k=0
for i, j in zip(madrid['Neighborhood'].sort_values(),zz.sort_values()):
    print(i,j,i==j)

Abrantes Abrantes True
Adelfas Adelfas True
Aeropuerto Aeropuerto True
Alameda de Osuna Alameda de Osuna True
Almagro Almagro True
Almenara Almenara True
Almendrales Almendrales True
Aluche Aluche True
Ambroz Ambroz True
Amposta Amposta True
Apostol Santiago Apostol Santiago True
Arapiles Arapiles True
Aravaca Aravaca True
Arcos Arcos True
Argüelles Argüelles True
Atalaya Atalaya True
Atocha Atocha True
Bellas Vistas Bellas Vistas True
Berruguete Berruguete True
Buenavista Buenavista True
Butarque Butarque True
Campamento Campamento True
Canillas Canillas True
Canillejas Canillejas True
Casa de Campo Casa de Campo True
Casco Histórico de Barajas Casco Histórico de Barajas True
Casco Histórico de Vallecas Casco Histórico de Vallecas True
Casco Histórico de Vicálvaro Casco Histórico de Vicálvaro True
Castellana Castellana True
Castilla Castilla True
Castillejos Castillejos True
Ciudad Jardín Ciudad Jardín True
Ciudad Universitaria Ciudad Universitaria True
Colina Colina True
Comillas Com

127

In [232]:
madrid[madrid['Neighborhood']=='Timón']

,District Name,Neighborhood,District Population,Net Income by Home,House Price 2019 €/m2,House Price 2018 €/m2,Neighborhood Latitude,Neighborhood Longitude
126,Barajas,Timón,46876,40.271,3088.0,0.0,40.474933,-3.600413


In [234]:
madrid['Neighborhood'][126]

'Timón'

,District Name,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,Palacio,40.416339,-3.714318,0,Plaza,Spanish Restaurant,Tapas Restaurant,Restaurant,Mexican Restaurant,Historic Site,Bar,Ramen Restaurant,Beer Bar,Music Venue
1,Centro,Embajadores,40.408129,-3.707739,0,Tapas Restaurant,Bar,Spanish Restaurant,Coffee Shop,Plaza,Art Gallery,Italian Restaurant,Outdoors & Recreation,Bistro,Café
2,Centro,Cortes,40.414221,-3.699941,0,Bar,Hotel,Hostel,Restaurant,Theater,Spanish Restaurant,Plaza,Tapas Restaurant,Mediterranean Restaurant,Café
3,Centro,Justicia,40.424531,-3.699293,0,Restaurant,Hotel,Spanish Restaurant,Bakery,Café,Mediterranean Restaurant,Bar,Pizza Place,Vegetarian / Vegan Restaurant,Bookstore
4,Centro,Universidad,40.423372,-3.707792,0,Hotel,Spanish Restaurant,Tapas Restaurant,Café,Restaurant,Cocktail Bar,Hostel,Theater,Bookstore,Argentinian Restaurant
5,Centro,Sol,40.416663,-3.704169,0,Hotel,Hostel,Plaza,Wine Bar,Gourmet Shop,Art Museum,Spanish Restaurant,Tapas Restaurant,Theater,Ice Cream Shop
6,Arganzuela,Imperial,40.406845,-3.720088,0,Spanish Restaurant,Grocery Store,Spa,Gym Pool,Park,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Café,Burger Joint
7,Arganzuela,Las Acacias,40.403926,-3.706549,1,Spanish Restaurant,Art Gallery,Pizza Place,Park,Bar,Bakery,Grocery Store,Tapas Restaurant,Sushi Restaurant,Theme Restaurant
8,Arganzuela,La Chopera,40.394836,-3.699599,0,Italian Restaurant,Tapas Restaurant,Park,Spanish Restaurant,Burger Joint,Art Gallery,Coffee Shop,Shopping Mall,Gastropub,Pet Store
9,Arganzuela,Legazpi,40.389234,-3.686412,0,Spanish Restaurant,Bar,Dessert Shop,Park,Supermarket,Restaurant,Coffee Shop,Multiplex,General Entertainment,Tapas Restaurant
